In [1]:
from azure_chat_openai_agent import AzureChatOpenAIAgent

agent = AzureChatOpenAIAgent
test_input = "What is fire safety?"
response = agent.converse("", test_input, "")

# Print the response
print("Response from AzureChatOpenAIAgent:", response)

ImportError: cannot import name 'ResponseSchema' from 'langchain_core.output_parsers' (c:\Users\choonho.ooi\AppData\Local\Programs\Python\Python313\Lib\site-packages\langchain_core\output_parsers\__init__.py)